## DCM Data length analysis

- 적절한 split length를 구하기 위해 법률 / 시행령 / 시행규칙에 따른 length를 알아보기

In [1]:
import os

import json
from llama_index.core import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


# JSON 파일을 로드하는 함수
def load_json_as_documents(input_file):
    # 파일을 열고 JSON 데이터를 파싱
    with open(input_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    # # schema에 맞추어 json flatten
    # flattened_json = [flatten_json(item) for item in json_data]

    documents = []; doc_length = []
    
    # JSON 리스트의 각 요소를 Document로 변환하여 리스트에 추가
    for entry in json_data:
        # 각 요소를 Document 객체로 변환
        doc = Document(
            text=entry["content"],
            metadata={
                "law_index": entry["index"],
                "name": entry["subtitle"],
                "document_title": entry["metadata"]["document_title"],
                "created_date": entry["metadata"]["date"],
                "revise_info": entry["metadata"]["revise_info"],
                "source": entry["metadata"]["source"],
                "title_doc": entry["metadata"]["title"]["doc"],
                "title_chapter": entry["metadata"]["title"]["chapter"],
                "title_section": entry["metadata"]["title"]["section"],
                "title_subsection": entry["metadata"]["title"]["subsection"],
            },
            metadata_seperator="::",
            metadata_template="{key}=>{value}",
            text_template="Metadata: {metadata_str}\n\nText: {content}",
        )
        documents.append(doc)
        doc_length.append(len(doc.text))
    
    return documents, doc_length


# 텍스트를 분할하는 함수
def split_documents(documents, chunk_size=1024):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=50,
        length_function=len
    )
    
    split_docs = []
    
    for doc in documents:
        # 텍스트를 분할하고 각 조각을 Document로 다시 생성
        chunks = text_splitter.split_text(doc.text)
        for chunk in chunks:
            split_doc = Document(
                text=chunk,
                metadata=doc.metadata,
                metadata_seperator=doc.metadata_seperator,
                metadata_template=doc.metadata_template,
                text_template=doc.text_template
            )
            split_docs.append(split_doc)
    
    return split_docs




### 법령

In [2]:
# 법령
import numpy as np

# JSON 파일 경로
input_file = "/mnt/c/Users/Shic/legal_graph/results/1-2/DCM_1-2_law_main_clause.json"

# 로더 사용하여 데이터 불러오기
documents, doc_length = load_json_as_documents(input_file)

# 문서를 1024 토큰 크기로 분할
splited_documents = split_documents(documents)

print(f"token length percentile : {np.percentile(doc_length, [0, 25, 50, 75, 100])}")
print(f"split 이전 개수 : {len(documents)}, split 이후 개수 : {len(splited_documents)}")

token length percentile : [  15.     77.    121.    204.25 4800.  ]
split 이전 개수 : 1968, split 이후 개수 : 1998


In [3]:
over_1024 = [x for x in doc_length if x >= 1024]
len(over_1024)

14

In [16]:
def find_jo(law_index, dataset):
    document_list = [x for x in dataset if x.metadata['law_index'] == law_index]
    print(f"{law_index} total chunk : ", len(document_list))

    for document in document_list:
        print("name : ", document.metadata['name'])
        print("title_doc : ", document.metadata['title_doc'])
        print("title_chapter : ", document.metadata['title_chapter'])
        print("title_section : ", document.metadata['title_section'])
        print("title_subsection : ", document.metadata['title_subsection'])
        print("text : ", document.text)

In [78]:
find_jo("제55조", splited_documents)

제55조 total chunk :  1
name :  손실보전 등의 금지
title_doc :  제2편 금융투자업
title_chapter :  제4장 영업행위 규칙
title_section :  제1절 공통 영업행위 규칙
title_subsection :  제3관 직무관련 정보의 이용 금지 등
text :  금융투자업자는 금융투자상품의 매매, 그 밖의 거래와 관련하여 제103조제3항에 따라 손실의 보전 또는 이익의 보장을 하는 경우, 그 밖에 건전한 거래질서를 해할 우려가 없는 경우로서 정당한 사유가 있는 경우를 제외하고는 다음 각 호의 어느 하나에 해당하는 행위를 하여서는 아니 된다. 금융투자업자의 임직원이 자기의 계산으로 하는 경우에도 또한 같다.
1. 투자자가 입을 손실의 전부 또는 일부를 보전하여 줄 것을 사전에 약속하는 행위
2. 투자자가 입은 손실의 전부 또는 일부를 사후에 보전하여 주는 행위
3. 투자자에게 일정한 이익을 보장할 것을 사전에 약속하는 행위
4. 투자자에게 일정한 이익을 사후에 제공하는 행위


### 시행령

In [6]:
# 시행령
import numpy as np

# JSON 파일 경로
input_file = "/mnt/c/Users/Shic/legal_graph/results/1-2/DCM_1-2_enforcement_main_clause.json"

# 로더 사용하여 데이터 불러오기
enforcement_documents, enforcement_doc_length = load_json_as_documents(input_file)

# 문서를 512 토큰 크기로 분할
enforcement_splited_documents = split_documents(enforcement_documents)

print(f"token length percentile : {np.percentile(enforcement_doc_length, [0, 25, 50, 75, 100])}")
print(f"split 이전 개수 : {len(enforcement_documents)}, split 이후 개수 : {len(enforcement_splited_documents)}")

token length percentile : [  15.   89.  166.  312. 9143.]
split 이전 개수 : 1648, split 이후 개수 : 1708


In [37]:
find_jo("제10조제2항", enforcement_splited_documents)

제10조제2항 total chunk :  1
name :  전문투자자의 범위 등
title_doc :  제1편 총칙
title_chapter :  None
title_section :  None
title_subsection :  None
text :  법 제9조제5항제3호에서 "대통령령으로 정하는 금융기관"이란 다음 각 호의 금융기관을 말한다.<개정 2009. 12. 21., 2010. 11. 15., 2012. 1. 6., 2015. 3. 3., 2016. 7. 28., 2016. 10. 25.>
1. 은행
2. 「한국산업은행법」에 따른 한국산업은행
3. 「중소기업은행법」에 따른 중소기업은행
4. 「한국수출입은행법」에 따른 한국수출입은행
5. 「농업협동조합법」에 따른 농업협동조합중앙회
6. 「수산업협동조합법」에 따른 수산업협동조합중앙회
7. 「보험업법」에 따른 보험회사(이하 "보험회사"라 한다)
8. 금융투자업자[법 제8조제9항에 따른 겸영금융투자업자(이하 "겸영금융투자업자"라 한다)는 제외한다]
9. 증권금융회사
10. 종합금융회사
11. 법 제355조제1항에 따라 인가를 받은 자금중개회사(이하 "자금중개회사"라 한다)
12. 「금융지주회사법」에 따른 금융지주회사
13. 「여신전문금융업법」에 따른 여신전문금융회사
14. 「상호저축은행법」에 따른 상호저축은행 및 그 중앙회
15. 「산림조합법」에 따른 산림조합중앙회
16. 「새마을금고법」에 따른 새마을금고연합회
17. 「신용협동조합법」에 따른 신용협동조합중앙회
18. 제1호부터 제17호까지의 기관에 준하는 외국 금융기관
